# Preprocessing brain MRI of patients with meningioma

## Before using MONSTR

### Setting PATH variable
**The environment PATH MUST be set in the main terminal, not otherwise like in a sub-terminal in the Jupyterlab**

#### ANTs
export ANTSPATH=/opt/ANTs/bin/

export PATH=${ANTSPATH}:$PATH

#### FSL
FSLDIR=/usr/local/fsl
. ${FSLDIR}/etc/fslconf/fsl.sh

PATH=${FSLDIR}/bin:${PATH}

export FSLDIR PATH


### Update
6/22 \
Code that applies brain mask added.

6/23 \
New variable T1C_label added\
(Sometimes you might want to do co-registration to sequence other than T1C, but this time we assume that you are  registering other sequences to T1C.)\

7/6 \
New variable mask_label added

7/7 \
Add try-except when finding T1Cpath and T2path using glob since T1C and T2 file might not exist

### Note
error \
terminate called after throwing an instance of 'std::invalid_argument' 

  what():  stoi

Aborted (core dumped)

==> due to space in filename

### to-do
get pt and seq using re function \
convert .ipynb to .py

In [5]:
import os
import shutil
import nibabel as nib
import numpy as np
from glob import glob

In [6]:
# make outputdir and return output path by combining outDir, pt, filename
def get_outpath(outDir, pt, filename):
    outPath = os.path.join(outDir, pt, filename)
    os.makedirs(os.path.join(outDir, pt), exist_ok=True)

    if os.path.exists(outPath):
        print('{} already exists'.format(filename))
        return None
    else:
        return outPath

# find sequence from filename
def find_seq(filename):
    seq_lst = ["T1_GD", "T1GD", "T1CE", "T1C", "CT1", "T1", "T2F", "3D_FLAIR", "FLAIRGD", "FLAIR", "T2", "F", "ADC", "BB", "WB"]
    seq = None
    for i in seq_lst:
        if i in filename.upper():
            seq = i
            return seq
    if seq is None:
        print("No matching sequence")
        return None

In [8]:
sourceDir = '/media/sc-mlsg/easystore/KHS_control_nii_sorted/'
targetDir = '/media/sc-mlsg/easystore/KHS_control_nii_sorted_preprocessed/'

#sourceDir = '/home/sc-mlsg/Downloads/Meningioma_nii/'
#targetDir = '/home/sc-mlsg/Downloads/Meningioma_nii_preprocessed/'

resampleDir_mask = os.path.join(targetDir, "00_Tumor_masks_resampled")
#registerDir_mask = os.path.join(targetDir, "00_Tumor_masks_registered")
resampleDir = os.path.join(targetDir, "01_Resampled")
correctionDir = os.path.join(targetDir, "02_Corrected")
registerDir = os.path.join(targetDir, '03_Registered_corrected')
brainmaskDir = os.path.join(targetDir, 'BrainMask_corrected')
strippedDir = os.path.join(targetDir, '04_Stripped_wo_Normalized')
znormedDir = os.path.join(targetDir, '05_Stripped_znormalized')

path_lst = [resampleDir, correctionDir, registerDir, brainmaskDir, strippedDir, znormedDir, resampleDir_mask]

for path in path_lst:
    os.makedirs(path, exist_ok=True)

T1C_label = 'T1C' # 'BB','T1C','T1GD' etc.
mask_label = 'mask' # '_mask', '-label' etc.

In [9]:
# check directory structure
for (path, dirs, files) in os.walk(sourceDir):
    print(path, dirs, files)

/home/sc-mlsg/Downloads/Meningioma_nii/ ['8155858', '7040064', '7350798'] []
/home/sc-mlsg/Downloads/Meningioma_nii/8155858 [] ['8155858_T1C.nii.gz', '8155858_T2.nii.gz']
/home/sc-mlsg/Downloads/Meningioma_nii/7040064 [] ['7040064_T1C.nii.gz', '7040064_T2.nii.gz']
/home/sc-mlsg/Downloads/Meningioma_nii/7350798 [] ['7350798_T1C.nii.gz', '7350798_T2.nii.gz']


In [10]:
# check file orientation
for (path, dirs, files) in os.walk(sourceDir):
    if files != []:
        for file in files:
            # load img
            img = nib.load(os.path.join(path, file))
            if nib.aff2axcodes(img.affine) != ('L', 'P', 'S'):
                print(file)
                print(nib.aff2axcodes(img.affine))
            

In [11]:
# 1) resampling
lst = ['postprocessing.json', 'volumes.txt', 'segmentation.nii.gz'] #,mask.nii.gz]
total = 0
for (path, dirs, files) in os.walk(sourceDir):
    if files != []:
        for file in files:
            if file not in lst:
                total += 1
                print("Resampling: #{}, {}".format(total, file))
                filePath = os.path.join(path, file)
                pt = path.split(os.path.sep)[-1]
                if mask_label in file:
                    pt = path.split(os.path.sep)[-1]
    #                pt = file.split("_")[0]
                    resample_outPath = get_outpath(resampleDir_mask, pt, file)
                else:
                    resample_outPath = get_outpath(resampleDir, pt, file)

                if resample_outPath is None:
                    continue
                !ResampleImage 3 {filePath} {resample_outPath} 1x1x1

Resampling: #1, 8155858_T1C.nii.gz
Resampling: #2, 8155858_T2.nii.gz
Resampling: #3, 7040064_T1C.nii.gz
Resampling: #4, 7040064_T2.nii.gz
Resampling: #5, 7350798_T1C.nii.gz
Resampling: #6, 7350798_T2.nii.gz


In [12]:
# 2) N4 bias correction
total = 0
for (path, dirs, files) in os.walk(resampleDir):
    for file in files:
        total += 1
        print("N4 bias correction: #{}, {}".format(total, file))
        filePath = os.path.join(path, file)
        pt = path.split(os.path.sep)[-1]
        corr_outPath = get_outpath(correctionDir, pt, file)
        if corr_outPath is None:
            continue
        !N4BiasFieldCorrection -d 3 -i {filePath} -o {corr_outPath} -s 3 -c [50x50x50x50,0.00001] -b [150]

N4 bias correction: #1, 8155858_T1C.nii.gz
N4 bias correction: #2, 8155858_T2.nii.gz
N4 bias correction: #3, 7040064_T1C.nii.gz
N4 bias correction: #4, 7040064_T2.nii.gz
N4 bias correction: #5, 7350798_T1C.nii.gz
N4 bias correction: #6, 7350798_T2.nii.gz


In [13]:
# 3) co-registration
total = 0
for (path, dirs, files) in os.walk(correctionDir):
    if files != []:
        total += 1
        print("coregistration: #{}, {}".format(total, files))
        pt = path.split(os.path.sep)[-1]
        
        try:
            T1C_path = glob(os.path.join(path, '*{}*'.format(T1C_label)))[-1]
        except:
            print("Target sequence not exists")
            continue
        T1C_regist_outPath = get_outpath(registerDir, pt, pt + '_{}_registered.nii'.format(T1C_label))
        if os.path.splitext(T1C_path)[-1] == '.gz':
            T1C_regist_outPath = get_outpath(registerDir, pt, pt + '_{}_registered.nii.gz'.format(T1C_label))

        if T1C_regist_outPath is not None:
            shutil.copyfile(T1C_path, T1C_regist_outPath)
        
        for file in files:
            seq = find_seq(file)
            if seq == T1C_label:
                continue
            in_path = os.path.join(path, file)
            print("Registering {} to {}...".format(seq, T1C_label))
            outPath = get_outpath(registerDir, pt, pt + '_{}_registered.nii.gz'.format(seq))
            if outPath is None:
                continue

            !antsRegistration -d 3 -r [{T1C_path}, {in_path}, 1] \
                              -m mattes[{T1C_path}, {in_path}, 1, 32, regular, 0.1 ] \
                              -t translation[0.1] -c [100x50x25,1.e-8,20] -s 4x2x1vox -f 6x4x2 -l 1 \
                              -m mattes[{T1C_path}, {in_path} , 1 , 32, regular, 0.1] \
                              -t rigid[0.1] -c [100x50x25,1.e-8,20]  -s 4x2x1vox  -f 3x2x1 -l 1 -o [reg, {outPath}] -v 0


coregistration: #1, ['8155858_T1C.nii.gz', '8155858_T2.nii.gz']
Registering T2 to T1C...
coregistration: #2, ['7040064_T1C.nii.gz', '7040064_T2.nii.gz']
Registering T2 to T1C...
coregistration: #3, ['7350798_T1C.nii.gz', '7350798_T2.nii.gz']
Registering T2 to T1C...


In [6]:
# 3) co-registration: mask(optional)
total = 0
for (path, dirs, files) in os.walk(resampleDir_mask):
    if files != []:
        total += 1
        print("coregistration: #{}, {}".format(total, files))
        pt = path.split(os.path.sep)[-1]

        try:
            T1C_path = glob(os.path.join(correctionDir, pt, '*{}*'.format(T1C_label)))[0]
        except:
            print("Target sequence not exists")
            continue
        print(T1C_path)
        for file in files:
            in_path = os.path.join(path, file)
            print("Registering mask to {}...".format(T1C_label))
            outPath = get_outpath(registerDir_mask, pt, file)
            if outPath is None:
                continue

            !antsRegistration -d 3 -r [{T1C_path}, {in_path}, 1] \
                              -m mattes[{T1C_path}, {in_path}, 1, 32, regular, 0.1 ] \
                              -t translation[0.1] -c [100x50x25,1.e-8,20] -s 4x2x1vox -f 6x4x2 -l 1 \
                              -m mattes[{T1C_path}, {in_path} , 1 , 32, regular, 0.1] \
                              -t rigid[0.1] -c [100x50x25,1.e-8,20]  -s 4x2x1vox  -f 3x2x1 -l 1 -o [reg, {outPath}] -v 0


coregistration: #1, ['8011358_mask.nii']
8011358
/media/sc-mlsg/Elements/20210322_IDHwtLGG_additional/Previous_patient/Severance_preprocessed/02_Corrected/8011358/8011358_T2F.nii.gz
Registering mask to T2F...
coregistration: #2, ['1371892_mask.nii']
1371892
/media/sc-mlsg/Elements/20210322_IDHwtLGG_additional/Previous_patient/Severance_preprocessed/02_Corrected/1371892/1371892_T2F.nii.gz
Registering mask to T2F...
coregistration: #3, ['2034571_mask.nii']
2034571
/media/sc-mlsg/Elements/20210322_IDHwtLGG_additional/Previous_patient/Severance_preprocessed/02_Corrected/2034571/2034571_T2F.nii.gz
Registering mask to T2F...
coregistration: #4, ['2174093_mask.nii']
2174093
Target sequence not exists
coregistration: #5, ['2545167_mask.nii']
2545167
/media/sc-mlsg/Elements/20210322_IDHwtLGG_additional/Previous_patient/Severance_preprocessed/02_Corrected/2545167/2545167_T2F.nii.gz
Registering mask to T2F...
coregistration: #6, ['2602057_mask.nii']
2602057
/media/sc-mlsg/Elements/20210322_IDHwtL

coregistration: #41, ['8604413_mask.nii']
8604413
/media/sc-mlsg/Elements/20210322_IDHwtLGG_additional/Previous_patient/Severance_preprocessed/02_Corrected/8604413/8604413_T2F.nii.gz
Registering mask to T2F...
coregistration: #42, ['8630035_mask.nii']
8630035
/media/sc-mlsg/Elements/20210322_IDHwtLGG_additional/Previous_patient/Severance_preprocessed/02_Corrected/8630035/8630035_T2F.nii.gz
Registering mask to T2F...
coregistration: #43, ['8633978_mask.nii']
8633978
/media/sc-mlsg/Elements/20210322_IDHwtLGG_additional/Previous_patient/Severance_preprocessed/02_Corrected/8633978/8633978_T2F.nii.gz
Registering mask to T2F...
coregistration: #44, ['8638813_mask.nii']
8638813
/media/sc-mlsg/Elements/20210322_IDHwtLGG_additional/Previous_patient/Severance_preprocessed/02_Corrected/8638813/8638813_T2F.nii.gz
Registering mask to T2F...
coregistration: #45, ['8640861_mask.nii']
8640861
/media/sc-mlsg/Elements/20210322_IDHwtLGG_additional/Previous_patient/Severance_preprocessed/02_Corrected/8640

In [26]:
# 4) Skull stripping using MONSTR

# we'll work in the MONSTR directory
workingDir = '/home/sc-mlsg/MONSTR'
%cd {workingDir} 

outputPath = '/home/sc-mlsg/MONSTR/output'
shutil.rmtree(outputPath)
os.makedirs(outputPath, exist_ok=True)

total = 0

for (path, dirs, files) in os.walk(registerDir):
    if files != []:
        total += 1
        print("skull stripping: #{}, {}".format(total, files))

        pt = path.split(os.path.sep)[-1]
        try:
            T1_path = glob(os.path.join(path, '*_T1_*'))[0]
        except:
            print("T1 not exists")
            continue
            
        try:
            T2_path = glob(os.path.join(path, '*T2*'))[0]
        except:
            print("T2 not exists")
            T2_path = None
            
        newFileName = pt + '_brain_mask.nii.gz'
        if os.path.exists(os.path.join(brainmaskDir, newFileName)):
            print("{} already exists".format(newFileName))
            continue

        !./MONSTR.sh --t1 {T1_path} --t2 {T2_path} --o {outputPath} --atlasdir TBI_atlas --reg

        newfile = glob(os.path.join(outputPath, '*{}*'.format(pt), 'atlas4_brainmask_ANTS.nii.gz'))[0]
        shutil.copyfile(newfile, os.path.join(brainmaskDir, newFileName))

/home/sc-mlsg/MONSTR
skull stripping: #1, ['8155858_T1C_registered.nii.gz', '8155858_T2_registered.nii.gz', '8155858_T1_registered.nii.gz']
8155858_brain_mask.nii.gz already exists
skull stripping: #2, ['7040064_T2_registered.nii.gz', '7040064_T1C_registered.nii.gz', '7040064_T1_registered.nii.gz']
I found ANTs installation at /opt/ANTs/bin 
Default number of atlases: 4 
Number of processors is not mentioned. Using all available 6 processors. 
Atlas directory : /home/sc-mlsg/MONSTR/TBI_atlas
T1 : /home/sc-mlsg/Downloads/Meningioma_nii_preprocessed/03_Registered_corrected/7040064/7040064_T1_registered.nii.gz
T2 : /home/sc-mlsg/Downloads/Meningioma_nii_preprocessed/03_Registered_corrected/7040064/7040064_T2_registered.nii.gz
Output directory : /home/sc-mlsg/MONSTR/output 
Temporary files will be kept.
Images are assumed to be registered. T2/PD/FLAIR will not be registered again to the T1. 
Working directory : /home/sc-mlsg/MONSTR/output/7040064_T1_registered.nii.conN
cd /home/sc-mlsg/MON

Interpolation type: BSplineInterpolateImageFunction
Output warped image: atlas4_T1_ANTS.nii
ANTS deformable registration took 0 HRS 1 MIN 28 SEC 
Using double precision for computations.
Input scalar image: /home/sc-mlsg/MONSTR/TBI_atlas/atlas4_T2.nii
Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas4_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas4_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: BSplineInterpolateImageFunction
Output warped image: atlas4_T2_ANTS.nii
Using double precision for computations.
Input scalar image: /home/sc-mlsg/MONSTR/TBI_atlas/atlas4_brainmask.nii
Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas4_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas4_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: NearestNeighborInterpol

Output warped image: atlas3_brainmask_ANTS.nii
AntsExample.sh t1.nii /home/sc-mlsg/MONSTR/TBI_atlas/atlas4_T1.nii fastfortesting atlas4_T1_ANTS.nii 6
Using double precision for computations.
Input scalar image: /home/sc-mlsg/MONSTR/TBI_atlas/atlas4_T1.nii
Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas4_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas4_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: BSplineInterpolateImageFunction
Output warped image: atlas4_T1_ANTS.nii
ANTS deformable registration took 0 HRS 1 MIN 35 SEC 
Using double precision for computations.
Input scalar image: /home/sc-mlsg/MONSTR/TBI_atlas/atlas4_T2.nii
Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas4_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas4_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel v

/home/sc-mlsg/MONSTR/image_dilate: error while loading shared libraries: libmwlaunchermain.so: cannot open shared object file: No such file or directory
invalid input dataset <t1_MultiModalStripMask_smooth.nii>
atlas1_brainmask_ANTS.nii:	 99.7% -- replaced with atlas1_brainmask_ANTS.nii.gz
atlas1_T1_ANTS.nii:	 10.1% -- replaced with atlas1_T1_ANTS.nii.gz
atlas1_T2_ANTS.nii:	  9.2% -- replaced with atlas1_T2_ANTS.nii.gz
atlas2_brainmask_ANTS.nii:	 99.7% -- replaced with atlas2_brainmask_ANTS.nii.gz
atlas2_T1_ANTS.nii:	 16.8% -- replaced with atlas2_T1_ANTS.nii.gz
atlas2_T2_ANTS.nii:	 16.1% -- replaced with atlas2_T2_ANTS.nii.gz
atlas3_brainmask_ANTS.nii:	 99.7% -- replaced with atlas3_brainmask_ANTS.nii.gz
atlas3_T1_ANTS.nii:	 18.9% -- replaced with atlas3_T1_ANTS.nii.gz
atlas3_T2_ANTS.nii:	 18.3% -- replaced with atlas3_T2_ANTS.nii.gz
atlas4_brainmask_ANTS.nii:	 99.8% -- replaced with atlas4_brainmask_ANTS.nii.gz
atlas4_T1_ANTS.nii:	 20.8% -- replaced with atlas4_T1_ANTS.nii.gz
atlas4_

In [27]:
# 5) apply skull mask
total = 0
error_dict = {}
for (path, dirs, files) in os.walk(registerDir):
    try:
        if files != []:
            total += 1
            pt = path.split(os.path.sep)[-1]
            mask = nib.load(os.path.join(brainmaskDir, pt + '_brain_mask.nii.gz'))

            for file in files:
                seq = find_seq(file)
                print("# {} Applying skull mask to {}...".format(total, file))
                outPath = get_outpath(strippedDir, pt, pt + "_{}_stripped.nii.gz".format(seq))
                if outPath is None:
                    continue
                
                file_nii = nib.load(os.path.join(path, file))
                masked_data = np.multiply(file_nii.get_data(), mask.get_data())
                masked_data = nib.Nifti1Image(masked_data, file_nii.affine, file_nii.header)
                nib.save(masked_data, outPath)
    except Exception as ex:
        print("Error:{}, filename:{}".format(ex.args, file))
        error_dict[pt] = ex.args

print("Finished!")
print(error_dict)

# 1 Applying skull mask to 8155858_T1C_registered.nii.gz...


/home/sc-mlsg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
/home/sc-mlsg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0


# 1 Applying skull mask to 8155858_T2_registered.nii.gz...
# 1 Applying skull mask to 8155858_T1_registered.nii.gz...
# 2 Applying skull mask to 7040064_T2_registered.nii.gz...
# 2 Applying skull mask to 7040064_T1C_registered.nii.gz...
# 2 Applying skull mask to 7040064_T1_registered.nii.gz...
# 3 Applying skull mask to 7350798_T1C_registered.nii.gz...
# 3 Applying skull mask to 7350798_T2_registered.nii.gz...
# 3 Applying skull mask to 7350798_T1_registered.nii.gz...
Finished!
{}


In [28]:
# 6) z-score normalization
total = 0
for (path, dirs, files) in os.walk(strippedDir):
    for file in files:
        try:
            total += 1
            print("Z-score normalization: #{}, {}".format(total, file))
            pt = path.split(os.path.sep)[-1]
            newfilename = file.split(".")[0] + "_znormed.nii.gz"
            znorm_outPath = get_outpath(znormedDir, pt, newfilename)
            if znorm_outPath is None:
                continue
            
            seq = find_seq(file)
            if seq == "ADC":
                print("ADC, copying file...")
                shutil.copyfile(os.path.join(path, file), znorm_outPath)
                continue
            img_nii = nib.load(os.path.join(path, file))
            img_data = img_nii.get_data()
            mean, std = np.mean(img_data), np.std(img_data)
            normalized = nib.Nifti1Image((img_data - mean) / std, img_nii.affine, img_nii.header)
            nib.save(normalized, znorm_outPath)
        except Exception as e:
            print(path, file)
            print(e)

Z-score normalization: #1, 8155858_T1_stripped.nii.gz


/home/sc-mlsg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0


Z-score normalization: #2, 8155858_T1C_stripped.nii.gz
Z-score normalization: #3, 8155858_T2_stripped.nii.gz
Z-score normalization: #4, 7040064_T1C_stripped.nii.gz
Z-score normalization: #5, 7040064_T2_stripped.nii.gz
Z-score normalization: #6, 7040064_T1_stripped.nii.gz
Z-score normalization: #7, 7350798_T2_stripped.nii.gz
Z-score normalization: #8, 7350798_T1_stripped.nii.gz
Z-score normalization: #9, 7350798_T1C_stripped.nii.gz
